### Initialize

In [148]:
%cd ~/PowerChain

import imp, web3
from pprint import pprint

CONTRACT_PATH = "./contracts/PowerChain.sol"
RPC_URL = "http://192.168.100.203:32301"

def red(text):
    return f"\x1b[31m{text}\x1b[0m"
def green(text):
    return f"\x1b[92m{text}\x1b[0m"
def result(bool):
    return green("PASS") if bool else red("FAIL")
def toAddress(address):
    return web3.Web3.to_checksum_address(address)
def toInternalNumber(number):
    return web3.Web3.to_wei(number,"ether")
def fromInternalNumber(number):
    return web3.Web3.from_wei(number,"ether")
def printResult(text,act,exp):
    res = exp == act
    print('{0:80}:  {1}'.format(text, result(res)))
    if (not res): raise Exception(f"Expected '{exp}' and got '{act}'")

/home/emmanouil/PowerChain


/home/emmanouil/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/emmanouil/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [149]:
#Create addresses to use with testing
admin = web3.Account.create()
user1 = web3.Account.create()
user2 = web3.Account.create()
user3 = web3.Account.create()
unit1 = web3.Account.create()
unit2 = web3.Account.create()
banker = web3.Account.create()

In [150]:
test = "Deploy contract successfully"
deployContract = imp.load_source('tools', './scripts/tools.py').deployContract
try:
    contractAddress = deployContract(RPC_URL,CONTRACT_PATH,admin)
    printResult(test,web3.Web3.is_address(contractAddress),True)
except Exception as e:
    print(e)
    printResult(test,False,True)

Smart Contract Deployed!
Contract Address: 0xC2062Be8576f0D6d355eF28f8F165AD7E1545411
[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': False,
              'internalType': 'string',
              'name': 'error',
              'type': 'string'}],
  'name': 'Error',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': False,
              'internalType': 'string',
              'name': 'info',
              'type': 'string'}],
  'name': 'Info',
  'type': 'event'},
 {'inputs': [{'internalType': 'address', 'name': 'addr', 'type': 'address'}],
  'name': 'addVoter',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'balanceENT',
  'outputs': [{'internalType': 'uint256',
               'name': 'available',
               'type': 'uint256'},
              {'internalType': 'uint256', 'name': 'locked', 'type': 'uint256'}],
  'stateMutability': 'nonpayable',
  '

### Load PowerChain Method Execution Encapsulation Class

In [151]:
PowerChain = imp.load_source('PowerChain', './scripts/pc_CallPowerChainMethod').PowerChain(
    "http://192.168.100.203:32301",
    "./contracts/PowerChain.sol.json" )

### Test Cases

Initial network values

In [152]:
rates = PowerChain.call("getEnergyRates")
printResult("Minting rate is 1 ENT/kWh", rates[0], 10**18)
printResult("Burning rate is 1 ENT/kWh", rates[1], 10**18)

print("Initial Network Parameter Values:")
parameters = PowerChain.call("getParameters")
print(f'''
Min Minting Rate                : {fromInternalNumber(parameters[0])} ENT/kWh
Max Burning Rate                : {fromInternalNumber(parameters[1])} ENT/kWh
Missing energy recover rate     : {fromInternalNumber(parameters[2])} ENT/kWh
Energy session validity period  : {parameters[3]} s
Storage Provider Fee            : {fromInternalNumber(parameters[4])} ENT per 1 ENT produced''')

Minting rate is 1 ENT/kWh                                                       :  PASS
Burning rate is 1 ENT/kWh                                                       :  PASS
Initial Network Parameter Values:

Min Minting Rate                : 0.01 ENT/kWh
Max Burning Rate                : 3 ENT/kWh
Missing energy recover rate     : 0.1 ENT/kWh
Energy session validity period  : 7200 s
Storage Provider Fee            : 0.2 ENT per 1 ENT produced


Test voter system

In [153]:
#Check if admin (user that deployed the contract) is an admin by default:
printResult("admin address is a voter by default", 
            PowerChain.call("isVoter",account=admin),True)

#user2 should not be a voter until admin upgrate it
printResult("user2 is not a voter initialy", 
            PowerChain.call("isVoter",account=user2), False)

#Admin (first voter) votes to make user2 a voter:
PowerChain.execute("addVoter",toAddress(user2.address),account=admin)

#Check if user2 became a voter:
printResult("user2 address upgrated with voter roles by admin", 
            PowerChain.call("isVoter",account=user2),True)

#Check if user is not a voter:
printResult("user1 address is not a voter", 
            PowerChain.call("isVoter",account=user1),False)

#Admin votes for user3 to become a voter:
PowerChain.execute("addVoter",
                   toAddress(user3.address),account=admin)
#user3 should not be a voter because now also user2 has to agree with this vote
printResult("Admin votes for user3 to became voter but also user2 has to agree now", 
            PowerChain.call("isVoter",account=user3),False)

#user2 votes for user3 to become a voter:
PowerChain.execute("addVoter",
                   toAddress(user3.address),account=user2)
#user3 should not be a voter because now also user2 has to agree with this vote
printResult("user2 votes for user3 to became a voter. Now user3 is a voter", 
            PowerChain.call("isVoter",account=user3), True)

#user3 votes to be removed from voter:
PowerChain.execute("removeVoter",
                   toAddress(user3.address),account=user3)
printResult("user3 votes to be removed from voter but 50% of voters should agree", 
            PowerChain.call("isVoter",account=user3), True)

#user2 votes to remove user3 from voter:
PowerChain.execute("removeVoter",
                   toAddress(user3.address),account=user2)
printResult("user2 votes to remove user3 from voter, user3 no longer a voter", 
            PowerChain.call("isVoter",account=user3), False)

#user2 votes to be removed from voter:
PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=user2)
printResult("user2 votes to be removed from voter but 50% of voters should agree", 
            PowerChain.call("isVoter",account=user2), True)

#user2 votes changes its vote, no longer want to be removed from voter:
PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=user2)
#admin votes to remove user2 from voter:
PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=admin)
printResult("user2 changes its vote and want to stay a voter. Admin votes to revome user2. user2 is still voter", 
            PowerChain.call("isVoter",account=user2), True)

#admin votes to remove user2 from voter:
PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=user2)
printResult("user2 votes to be removed from voter again and is no longer a voter", 
            PowerChain.call("isVoter",account=user2), False)

admin address is a voter by default                                             :  PASS
user2 is not a voter initialy                                                   :  PASS
user2 address upgrated with voter roles by admin                                :  PASS
user1 address is not a voter                                                    :  PASS
Admin votes for user3 to became voter but also user2 has to agree now           :  PASS
user2 votes for user3 to became a voter. Now user3 is a voter                   :  PASS
user3 votes to be removed from voter but 50% of voters should agree             :  PASS
user2 votes to remove user3 from voter, user3 no longer a voter                 :  PASS
user2 votes to be removed from voter but 50% of voters should agree             :  PASS
user2 changes its vote and want to stay a voter. Admin votes to revome user2. user2 is still voter:  PASS
user2 votes to be removed from voter again and is no longer a voter             :  PASS


In [162]:
#Admin votes
votes = [("Vote String","User Vote", "Final Decision")]
votes.append(PowerChain.call("getVotes",account=admin))
pprint(votes)

[('Vote String', 'User Vote', 'Final Decision'),
 [('Add_Voter_210e2f8b2a18c410e622266f2f49794d8370c694', True, True),
  ('Add_Voter_52d0056d999c5af65f72c023a8d5389fcd3058a0', True, True),
  ('Remove_Voter_210e2f8b2a18c410e622266f2f49794d8370c694', True, True),
  ('register_64cd7c429a26825ebdeb90c86127b0587a2f62aa_9aa52c778d8596224b6174ef4b8870b687a205e9',
   True,
   True),
  ('Add_Voter_210e2f8b2a18c410e622266f2f49794d8370c694_1', True, True),
  ('register_d2dc84d057ab60b7b8fbb41cbb19856d5d192ab5_52d0056d999c5af65f72c023a8d5389fcd3058a0',
   True,
   True),
  ('Remove_Voter_210e2f8b2a18c410e622266f2f49794d8370c694_1', True, True),
  ('remove_d2dc84d057ab60b7b8fbb41cbb19856d5d192ab5', True, True),
  ('remove_64cd7c429a26825ebdeb90c86127b0587a2f62aa', True, True)]]


Test Energy Management System

In [163]:
printResult("Total energy in the network is initially zero",
            PowerChain.call("getTotalEnergy"),0)

printResult("Total ENT in the network is initially zero",
            PowerChain.call("getTotalENT"),0)

printResult("user1 has zero ENT tokens",
            PowerChain.call("balanceENT",account=user1), [0, 0])

printResult("There are no storage units in the network yet",
            PowerChain.call("getStorageUnits"),[])

printResult("Unit1 fails to produce kWhs and mint ENT tokens as it's not yet a storage unit",
            PowerChain.execute("energyProduced",toAddress(unit2.address),123,account=unit1) != "",True)

printResult("Unit1 fails to consume kWhs and burn ENT tokens  as it's not yet a storage unit",
            PowerChain.execute("energyConsumed",toAddress(unit2.address),321,account=unit1) != "", True)

PowerChain.execute("registerStorageUnit",toAddress(unit1.address),toAddress(user1.address),account=admin)
printResult("Admin makes unit1 a storage unit with owner user1",
            unit1.address in PowerChain.call("getStorageUnits"), True)

PowerChain.execute("addVoter",toAddress(user2.address),account=admin)
printResult("Admin makes user2 a voter",
            PowerChain.call("isVoter",account=user2),True)

PowerChain.execute("registerStorageUnit",toAddress(unit2.address),toAddress(user3.address),account=admin)
printResult("Admin votes to make unit2 a storage unit with owner user3. user2 has to agree.",
            unit2.address not in PowerChain.call("getStorageUnits"), True)

PowerChain.execute("registerStorageUnit",toAddress(unit2.address),toAddress(user3.address),account=user2)
storageUnits = PowerChain.call("getStorageUnits")
printResult("User2 votes to make unit2 a storage unit with owner user3. It's now a storage unit",
            unit1.address in storageUnits and unit2.address in storageUnits, True)

PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=user2)
PowerChain.execute("removeVoter",
                   toAddress(user2.address),account=admin)
printResult("User2 and Admin votes to remove user2 from voters and is no longer a voter",
            not PowerChain.call("isVoter",account=user2), True)

printResult("User2 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute("energyProduced",toAddress(user2.address),10000,account=unit1), "")

printResult("Network has 10 kWh in total",
            PowerChain.call("getTotalEnergy"), 10000)

printResult("User2 has now 8 ENT (20% is the storage provider fee)",
            PowerChain.call("balanceENT",account=user2), [toInternalNumber(8), 0])

printResult("User1 (owner of unit1 storage unit) has now 2 ENT",
            PowerChain.call("balanceENT",account=user1), [toInternalNumber(2), 0])

printResult("Network has 10 ENT in total",
            PowerChain.call("getTotalENT",account=user1), toInternalNumber(10))

printResult("User1 starts a consumption session worth 2 ENT with unit1",
            "session started" in PowerChain.execute("startConsumptionSession",toAddress(unit1.address),toInternalNumber(2),account=user1),True)

printResult("User1 has now one active consumption sessions",
            len(PowerChain.call("getConsumptionSessions",account=user1)), 1)

printResult("Unit1 checks consumption session and has 2kWh",
            PowerChain.call("getConsumptionSessionEnergy",toAddress(user1.address),account=unit1),2000)

printResult("User1 has now 2 ENT locked",
            PowerChain.call("balanceENT",account=user1),[0,toInternalNumber(2)])

printResult("User1 consumes 1kWh from the consumption session and unit1 reports it",
            PowerChain.execute("energyConsumed",toAddress(user1.address),1000,account=unit1),"")

printResult("User1 has now 1 locked ENT",
            PowerChain.call("balanceENT",account=user1),[0,toInternalNumber(1)])

printResult("User1 checks consumption session and has now 1 kWh",
            PowerChain.call("getConsumptionSessionEnergy",toAddress(unit1.address),account=user1), 1000)

printResult("User1 consumes 1kWh from the consumption session and unit1 reports it",
            PowerChain.execute("energyConsumed",toAddress(user1.address),1000,account=unit1), "")

printResult("User1 has no longer any active consumption sessions",
            PowerChain.call("getConsumptionSessions",account=user1), [])

printResult("Unit1 has now 8 kWh",
            PowerChain.call("getStorageUnitEnergy",toAddress(unit1.address)),8000)

printResult("Total network energy is 8 kWh",
            PowerChain.call("getTotalEnergy"),8000)

printResult("User3 produces 2 kWh to unit2 and unit2 reports the production",
            PowerChain.execute("energyProduced",toAddress(user3.address),2000,account=unit2), "" )

printResult("User3 has now 2 ENT",
            PowerChain.call("balanceENT",account=user3),[toInternalNumber(2),0])

printResult("User3 start a consumption session with unit2 worth of 3 ENT but fails because it has 2",
            "session started" not in PowerChain.execute("startConsumptionSession",toAddress(unit2.address),toInternalNumber(3),account=user3),True)

printResult("There is no consumption session for user3",
            PowerChain.call("getConsumptionSessions",account=user3), [])

printResult("User3 transfers 2 ENT to user2",
            PowerChain.execute("transferENT",toAddress(user2.address),toInternalNumber(2),account=user3), "")

printResult("User2 has now 10 ENT",
            PowerChain.call("balanceENT",account=user2),[toInternalNumber(10),0])

printResult("User2 start a consumption session with unit2 worth of 3 ENT but unit2 don't have enough energy",
            "session started" not in PowerChain.execute("startConsumptionSession",toAddress(unit2.address),toInternalNumber(3),account=user2),True)

printResult("There is no consumption session for user2",
            PowerChain.call("getConsumptionSessions",account=user2), [])

printResult("User2 start a consumption session with unit2 worth of 2 ENT",
            "session started" in PowerChain.execute("startConsumptionSession",toAddress(unit2.address),toInternalNumber(2),account=user2),True)

printResult("There is one consumption session for user2",
            len(PowerChain.call("getConsumptionSessions",account=user2)), 1)

printResult("User2 start a consumption session with unit1 worth of 8 ENT",
            "session started" in PowerChain.execute("startConsumptionSession",toAddress(unit1.address),toInternalNumber(8),account=user2), True)

printResult("There are two consumption sessions for user2",
            len(PowerChain.call("getConsumptionSessions",account=user2)), 2)

printResult("User2 consumes 2kWh from the consumption session with unit2 and unit2 reports it",
            PowerChain.execute("energyConsumed",toAddress(user2.address),2000,account=unit2) , "")

printResult("There should be one consumption session now for user2",
            len(PowerChain.call("getConsumptionSessions",account=user2)) , 1)

PowerChain.execute("removeStorageUnit",toAddress(unit2.address),account=admin)
printResult("Admin votes to remove unit2 from storage units and it's no longer a storage unit",
            unit2.address not in PowerChain.call("getStorageUnits"),True)

printResult("User1 consumes 9kWh (over consumption, 1kwh more) from session with unit1 and unit1 reports it",
            PowerChain.execute("energyConsumed",toAddress(user2.address),9000,account=unit1) , "")

printResult("There should be no consumption session now for user2",
            PowerChain.call("getConsumptionSessions",account=user2), [])

PowerChain.execute("removeStorageUnit",toAddress(unit1.address),account=admin)
printResult("Admin votes to remove unit1 from storage units and it's no longer a storage unit",
            unit1.address not in PowerChain.call("getStorageUnits"),True)

Total energy in the network is initially zero                                   :  PASS
Total ENT in the network is initially zero                                      :  PASS
user1 has zero ENT tokens                                                       :  PASS
There are no storage units in the network yet                                   :  PASS
Unit1 fails to produce kWhs and mint ENT tokens as it's not yet a storage unit  :  PASS
Unit1 fails to consume kWhs and burn ENT tokens  as it's not yet a storage unit :  PASS
Admin makes unit1 a storage unit with owner user1                               :  PASS
Admin makes user2 a voter                                                       :  PASS
Admin votes to make unit2 a storage unit with owner user3. user2 has to agree.  :  PASS
User2 votes to make unit2 a storage unit with owner user3. It's now a storage unit:  PASS
User2 and Admin votes to remove user2 from voters and is no longer a voter      :  PASS
User2 produces 10 kWh in unit1

Test handling of energy imbalance

In [ ]:
printResult("Minting rate is 1 ENT/kWh", rates[0], toInternalNumber(1))
printResult("Burning rate is 1 ENT/kWh", rates[1], toInternalNumber(1))

params = PowerChain.call("getParameters")
c = params[2]

PowerChain.execute("registerStorageUnit",toAddress(unit1.address),toAddress(user1.address),account=admin)
printResult("Admin makes unit1 a storage unit with owner user1",
            unit1.address in PowerChain.call("getStorageUnits"), True)

printResult("User1 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute("energyProduced",toAddress(user1.address),10000,account=unit1), "")

printResult("User1 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute("energyProduced",toAddress(user1.address),10000,account=unit1), "")

printResult("User1 starts a consumption session worth 8 ENT with unit1",
            "session started" in PowerChain.execute("startConsumptionSession",toAddress(unit1.address),toInternalNumber(8),account=user1),True)

printResult("User1 checks consumption session and has 8 kWh",
            PowerChain.call("getConsumptionSessionEnergy",toAddress(user1.address),account=unit1),8000)

printResult("Something happened to a battery and unit 1 reports that it actually has 9 kWh",
            PowerChain.execute("reportActualEnergy",9000,account=unit1), "")

printResult("Minting rate is now 0.9 ENT/kWh", rates[0], toInternalNumber(0.9))
printResult("Burning rate is now 1.1 ENT/kWh", rates[1], toInternalNumber(1.1))

printResult("User1 checks consumption session and has still 8 kWh as there is enough energy",
            PowerChain.call("getConsumptionSessionEnergy",toAddress(unit1.address),account=user1),8000)

printResult("User1 has 8 ENT locked and 2 ENT unlocked",
            PowerChain.call("balanceENT",account=user2),[toInternalNumber(2),toInternalNumber(8)])

printResult("User1 consumes 8 kWh and unit1 reports it",
            PowerChain.execute("energyConsumed",toAddress(user1.address),8000,account=unit1) , "")

printResult("There are no consumption session now for user1",
            PowerChain.call("getConsumptionSessions",account=user1), [])

printResult("User1 has now 2 ENT",
            PowerChain.call("balanceENT",account=user1),[toInternalNumber(2),0])

printResult("Total network energy is 1 kWh",
            PowerChain.call("getTotalEnergy"),1000)

printResult("Network has 2 ENT in total",
            PowerChain.call("getTotalENT"), toInternalNumber(2))

printResult("User1 starts a consumption session worth 1.1 ENT with unit1",
            "session started" in PowerChain.execute("startConsumptionSession",toAddress(unit1.address),toInternalNumber(1.1),account=user1),True)

printResult("User1 checks consumption session and has 1 kWh",
            PowerChain.call("getConsumptionSessionEnergy",toAddress(unit1.address),account=user1),1000)

printResult("Minting rate is 1 ENT/kWh", rates[0], toInternalNumber(0.91))
printResult("Burning rate is 1 ENT/kWh", rates[1], toInternalNumber(1.09))
